In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from scipy.ndimage import convolve

print("Lattice Gas Automaton: From Microdynamics to Macrodynamics")
print("=" * 60)

Lattice Gas Automaton: From Microdynamics to Macrodynamics


In [2]:
# ===================================================================
# PART 1: HEXAGONAL LATTICE GEOMETRY
# ===================================================================

class HexLattice:
    """
    Hexagonal lattice geometry for lattice gas.
    6 velocity directions at 60° intervals.
    """
    
    def __init__(self):
        # 6 velocity vectors (unit speed)
        # c_i = (cos(π*i/3), sin(π*i/3)) for i = 0,1,2,3,4,5
        self.num_velocities = 6
        angles = np.array([i * np.pi / 3 for i in range(6)])
        
        self.cx = np.cos(angles)
        self.cy = np.sin(angles)
        
        # Store as integer displacements for lattice movement
        # (rounded to nearest integer for discrete lattice)
        self.c = np.array([self.cx, self.cy]).T
        
        # Opposite directions (for collision rules)
        self.opposite = np.array([3, 4, 5, 0, 1, 2])
        
    def get_velocities(self):
        """Return velocity vectors"""
        return self.c
    
    def visualize_directions(self):
        """Visualize the 6 velocity directions"""
        fig = go.Figure()
        
        # Draw hexagon
        hex_angles = np.linspace(0, 2*np.pi, 7)
        hex_x = np.cos(hex_angles)
        hex_y = np.sin(hex_angles)
        
        fig.add_trace(go.Scatter(
            x=hex_x, y=hex_y,
            mode='lines',
            line=dict(color='lightgray', width=2),
            name='Lattice cell',
            showlegend=False
        ))
        
        # Draw velocity directions
        colors = px.colors.qualitative.Set1
        for i in range(6):
            fig.add_annotation(
                x=0, y=0,
                ax=self.cx[i]*0.8, ay=self.cy[i]*0.8,
                xref='x', yref='y',
                axref='x', ayref='y',
                showarrow=True,
                arrowhead=2,
                arrowsize=1.5,
                arrowwidth=3,
                arrowcolor=colors[i],
            )
            # Label
            fig.add_trace(go.Scatter(
                x=[self.cx[i]*1.1],
                y=[self.cy[i]*1.1],
                mode='text',
                text=[f'c{i}'],
                textfont=dict(size=14, color=colors[i]),
                showlegend=False
            ))
        
        # Center point
        fig.add_trace(go.Scatter(
            x=[0], y=[0],
            mode='markers',
            marker=dict(size=10, color='black'),
            showlegend=False
        ))
        
        fig.update_layout(
            title='Hexagonal Lattice: 6 Velocity Directions',
            xaxis=dict(range=[-1.5, 1.5], scaleanchor='y', scaleratio=1),
            yaxis=dict(range=[-1.5, 1.5]),
            template='plotly_white',
            height=500,
            width=500
        )
        
        fig.show()

# Create and visualize
lattice = HexLattice()
lattice.visualize_directions()

In [3]:
# ===================================================================
# PART 2: COLLISION RULES
# ===================================================================

class CollisionRules:
    """
    Implements FHP (Frisch-Hasslacher-Pomeau) collision rules.
    Conserves mass and momentum.
    """
    
    def __init__(self, lattice):
        self.lattice = lattice
        self.num_velocities = lattice.num_velocities
        
    def two_body_collision(self, n):
        """
        Two-body head-on collision.
        Before: particles at i and i+3 (opposite directions)
        After: particles at i+1 and i+4 (or i+2 and i+5)
        
        Parameters:
        -----------
        n : array of shape (6,)
            Occupation numbers at a single site
            
        Returns:
        --------
        n_new : array of shape (6,)
            Occupation numbers after collision
        """
        n_new = n.copy()
        
        # Check all three possible head-on pairs
        for i in range(3):
            j = (i + 3) % 6  # Opposite direction
            
            # If exactly these two channels are occupied
            if n[i] == 1 and n[j] == 1 and np.sum(n) == 2:
                # Scatter at 60° with 50% probability
                if np.random.rand() < 0.5:
                    # Scatter to i+1 and i+4
                    i1 = (i + 1) % 6
                    i2 = (i + 4) % 6
                    if n[i1] == 0 and n[i2] == 0:
                        n_new[i] = 0
                        n_new[j] = 0
                        n_new[i1] = 1
                        n_new[i2] = 1
                        return n_new
                else:
                    # Scatter to i+2 and i+5
                    i1 = (i + 2) % 6
                    i2 = (i + 5) % 6
                    if n[i1] == 0 and n[i2] == 0:
                        n_new[i] = 0
                        n_new[j] = 0
                        n_new[i1] = 1
                        n_new[i2] = 1
                        return n_new
        
        return n_new
    
    def three_body_collision(self, n):
        """
        Three-body collision (alternating pattern).
        Before: particles at i, i+2, i+4 (every other direction)
        After: particles at i+1, i+3, i+5
        
        Parameters:
        -----------
        n : array of shape (6,)
            Occupation numbers at a single site
            
        Returns:
        --------
        n_new : array of shape (6,)
            Occupation numbers after collision
        """
        n_new = n.copy()
        
        # Check if we have alternating pattern (0,2,4) or (1,3,5)
        pattern1 = (n[0]==1 and n[2]==1 and n[4]==1 and 
                   n[1]==0 and n[3]==0 and n[5]==0)
        pattern2 = (n[1]==1 and n[3]==1 and n[5]==1 and 
                   n[0]==0 and n[2]==0 and n[4]==0)
        
        if pattern1:
            # Swap to pattern 2
            n_new = np.array([0, 1, 0, 1, 0, 1])
        elif pattern2:
            # Swap to pattern 1
            n_new = np.array([1, 0, 1, 0, 1, 0])
            
        return n_new
    
    def apply_collision(self, n):
        """
        Apply collision rules to a single site.
        
        Parameters:
        -----------
        n : array of shape (6,)
            Occupation numbers
            
        Returns:
        --------
        n_new : array of shape (6,)
            Occupation numbers after collision
        """
        # Try three-body first (less common)
        n_new = self.three_body_collision(n)
        if not np.array_equal(n_new, n):
            return n_new
        
        # Try two-body
        n_new = self.two_body_collision(n)
        return n_new
    
    def visualize_collisions(self):
        """Visualize example collision rules"""
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=[
                'Two-body: Before',
                'Two-body: After',
                'Three-body: Before',
                'Three-body: After'
            ],
            specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
                   [{'type': 'scatter'}, {'type': 'scatter'}]]
        )
        
        def draw_site(fig, n, row, col, title=''):
            """Draw a single site with occupied channels"""
            cx = self.lattice.cx
            cy = self.lattice.cy
            
            # Draw all directions (gray for empty)
            for i in range(6):
                color = 'red' if n[i] == 1 else 'lightgray'
                width = 4 if n[i] == 1 else 1
                
                fig.add_annotation(
                    x=0, y=0,
                    ax=cx[i]*0.6, ay=cy[i]*0.6,
                    xref=f'x{col}', yref=f'y{row}',
                    axref=f'x{col}', ayref=f'y{row}',
                    showarrow=True,
                    arrowhead=2,
                    arrowwidth=width,
                    arrowcolor=color,
                    row=row, col=col
                )
            
            # Center point
            fig.add_trace(go.Scatter(
                x=[0], y=[0],
                mode='markers',
                marker=dict(size=8, color='black'),
                showlegend=False
            ), row=row, col=col)
        
        # Two-body collision example
        n_before_2 = np.array([1, 0, 0, 1, 0, 0])  # Head-on at 0 and 3
        n_after_2 = np.array([0, 1, 0, 0, 1, 0])   # Scattered to 1 and 4
        
        draw_site(fig, n_before_2, 1, 1)
        draw_site(fig, n_after_2, 1, 2)
        
        # Three-body collision example
        n_before_3 = np.array([1, 0, 1, 0, 1, 0])  # Alternating pattern
        n_after_3 = np.array([0, 1, 0, 1, 0, 1])   # Swapped pattern
        
        draw_site(fig, n_before_3, 2, 1)
        draw_site(fig, n_after_3, 2, 2)
        
        # Update layout
        for i in range(1, 3):
            for j in range(1, 3):
                fig.update_xaxes(range=[-1, 1], showgrid=False, zeroline=False, 
                               showticklabels=False, row=i, col=j)
                fig.update_yaxes(range=[-1, 1], showgrid=False, zeroline=False,
                               showticklabels=False, scaleanchor=f'x{j}', 
                               scaleratio=1, row=i, col=j)
        
        fig.update_layout(
            title_text='Collision Rules: Mass and Momentum Conserving',
            template='plotly_white',
            height=600,
            showlegend=False
        )
        
        fig.show()

# Visualize collision rules
collision_rules = CollisionRules(lattice)
collision_rules.visualize_collisions()

In [5]:
# ===================================================================
# PART 3: LATTICE GAS SIMULATION (MICROSCOPIC)
# ===================================================================

class LatticeGas:
    """
    Full lattice gas automaton simulation on a 2D grid.
    Uses a simplified square lattice with hexagonal-like velocity structure.
    """
    
    def __init__(self, Nx, Ny, lattice, collision_rules):
        """
        Parameters:
        -----------
        Nx, Ny : int
            Lattice dimensions
        lattice : HexLattice
            Lattice geometry
        collision_rules : CollisionRules
            Collision rules
        """
        self.Nx = Nx
        self.Ny = Ny
        self.lattice = lattice
        self.collision_rules = collision_rules
        self.num_velocities = lattice.num_velocities
        
        # State: occupation numbers n_i(x,y,t)
        # Shape: (Nx, Ny, 6)
        self.n = np.zeros((Nx, Ny, self.num_velocities), dtype=int)
        
    def initialize_random(self, density=0.3):
        """Initialize with random particles"""
        self.n = (np.random.rand(self.Nx, self.Ny, self.num_velocities) < density).astype(int)
        
    def initialize_shear_flow(self, density=0.5):
        """Initialize with a shear flow pattern"""
        self.n = np.zeros((self.Nx, self.Ny, self.num_velocities), dtype=int)
        
        for i in range(self.Nx):
            for j in range(self.Ny):
                # Top half flows right, bottom half flows left
                if j > self.Ny // 2:
                    if np.random.rand() < density:
                        self.n[i, j, 0] = 1  # Right
                else:
                    if np.random.rand() < density:
                        self.n[i, j, 3] = 1  # Left
    
    def initialize_obstacle(self, density=0.3, obstacle_center=None, obstacle_radius=5):
        """Initialize with uniform flow and a circular obstacle"""
        self.n = np.zeros((self.Nx, self.Ny, self.num_velocities), dtype=int)
        
        if obstacle_center is None:
            obstacle_center = (self.Nx // 4, self.Ny // 2)
        
        for i in range(self.Nx):
            for j in range(self.Ny):
                # Check if inside obstacle
                dx = i - obstacle_center[0]
                dy = j - obstacle_center[1]
                if dx**2 + dy**2 < obstacle_radius**2:
                    continue  # No particles in obstacle
                
                # Uniform rightward flow
                if np.random.rand() < density:
                    self.n[i, j, 0] = 1  # Right direction
    
    def propagation_step(self):
        """
        Propagation: particles move to neighboring sites.
        n_i(x + c_i, t+1) = n_i(x, t) + Δ_i
        """
        n_new = np.zeros_like(self.n)
        
        # For simplicity on square lattice, we approximate hexagonal movement
        # Using a simplified 6-direction scheme
        moves = [
            (1, 0),    # Right (0)
            (1, 1),    # Upper-right (1)
            (0, 1),    # Up (2)
            (-1, 0),   # Left (3)
            (-1, -1),  # Lower-left (4)
            (0, -1)    # Down (5)
        ]
        
        for i in range(self.num_velocities):
            dx, dy = moves[i]
            # Roll the array (periodic boundary conditions)
            n_new[:, :, i] = np.roll(np.roll(self.n[:, :, i], dx, axis=0), dy, axis=1)
        
        self.n = n_new
    
    def collision_step(self):
        """
        Collision: apply collision rules at each site.
        """
        for i in range(self.Nx):
            for j in range(self.Ny):
                self.n[i, j, :] = self.collision_rules.apply_collision(self.n[i, j, :])
    
    def step(self):
        """One full time step: collision then propagation"""
        self.collision_step()
        self.propagation_step()
    
    def get_density(self):
        """Calculate local density ρ(x,y) = Σ_i n_i(x,y)"""
        return np.sum(self.n, axis=2)
    
    def get_velocity(self):
        """
        Calculate local velocity u(x,y) = Σ_i c_i n_i(x,y) / ρ(x,y)
        Returns ux, uy
        """
        ux = np.zeros((self.Nx, self.Ny))
        uy = np.zeros((self.Nx, self.Ny))
        
        for i in range(self.num_velocities):
            ux += self.lattice.cx[i] * self.n[:, :, i]
            uy += self.lattice.cy[i] * self.n[:, :, i]
        
        rho = self.get_density()
        # Avoid division by zero
        mask = rho > 0
        ux[mask] /= rho[mask]
        uy[mask] /= rho[mask]
        
        return ux, uy
    
    def get_momentum(self):
        """Calculate total momentum"""
        ux, uy = self.get_velocity()
        rho = self.get_density()
        return np.sum(rho * ux), np.sum(rho * uy)
    
    def visualize_microscopic(self, title='Microscopic View'):
        """Visualize individual particles"""
        fig = go.Figure()
        
        # Show particles as arrows
        for i in range(self.num_velocities):
            x_coords = []
            y_coords = []
            u_coords = []
            v_coords = []
            
            for ix in range(self.Nx):
                for iy in range(self.Ny):
                    if self.n[ix, iy, i] == 1:
                        x_coords.append(ix)
                        y_coords.append(iy)
                        u_coords.append(self.lattice.cx[i] * 0.4)
                        v_coords.append(self.lattice.cy[i] * 0.4)
            
            if len(x_coords) > 0:
                # Use annotations for arrows (small scale)
                for x, y, u, v in zip(x_coords, y_coords, u_coords, v_coords):
                    fig.add_annotation(
                        x=x, y=y,
                        ax=x+u, ay=y+v,
                        xref='x', yref='y',
                        axref='x', ayref='y',
                        showarrow=True,
                        arrowhead=2,
                        arrowsize=1,
                        arrowwidth=2,
                        arrowcolor='blue'
                    )
        
        fig.update_layout(
            title=title,
            xaxis=dict(range=[-1, self.Nx], title='x'),
            yaxis=dict(range=[-1, self.Ny], title='y', scaleanchor='x', scaleratio=1),
            template='plotly_white',
            height=600,
            width=800
        )
        
        fig.show()
    
    def visualize_macroscopic(self, title='Macroscopic View'):
        """Visualize density and velocity fields"""
        rho = self.get_density()
        ux, uy = self.get_velocity()
        
        # Subsample for quiver plot
        skip = max(1, self.Nx // 20)
        X, Y = np.meshgrid(range(0, self.Nx, skip), range(0, self.Ny, skip))
        U = ux[::skip, ::skip].T
        V = uy[::skip, ::skip].T
        
        fig = go.Figure()
        
        # Density heatmap
        fig.add_trace(go.Heatmap(
            z=rho.T,
            colorscale='Viridis',
            name='Density',
            colorbar=dict(title='ρ')
        ))
        
        # Velocity field (quiver plot approximation)
        fig.add_trace(go.Scatter(
            x=X.flatten(),
            y=Y.flatten(),
            mode='markers',
            marker=dict(size=1, color='white'),
            showlegend=False,
            hoverinfo='skip'
        ))
        
        # Add arrows
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                if U[i,j]**2 + V[i,j]**2 > 0.01:  # Only show significant velocities
                    fig.add_annotation(
                        x=X[i,j], y=Y[i,j],
                        ax=X[i,j] + U[i,j]*skip*0.8,
                        ay=Y[i,j] + V[i,j]*skip*0.8,
                        xref='x', yref='y',
                        axref='x', ayref='y',
                        showarrow=True,
                        arrowhead=2,
                        arrowsize=1,
                        arrowwidth=2,
                        arrowcolor='white'
                    )
        
        fig.update_layout(
            title=title,
            xaxis=dict(title='x'),
            yaxis=dict(title='y', scaleanchor='x', scaleratio=1),
            template='plotly_white',
            height=600,
            width=800
        )
        
        fig.show()

print("\n=== Example 1: Small System Microscopic View ===")


=== Example 1: Small System Microscopic View ===


In [6]:
# Small system for microscopic visualization
lg_small = LatticeGas(Nx=20, Ny=20, lattice=lattice, collision_rules=collision_rules)
lg_small.initialize_random(density=0.2)

print("Initial state:")
lg_small.visualize_microscopic(title='Microscopic View: Initial Random State')

Initial state:


In [7]:
# Evolve for a few steps
print("\nAfter 10 steps:")
for _ in range(10):
    lg_small.step()
lg_small.visualize_microscopic(title='Microscopic View: After 10 Steps')


After 10 steps:


In [8]:
print("\n=== Example 2: Larger System Macroscopic View ===")

# Larger system for macroscopic behavior
lg_large = LatticeGas(Nx=80, Ny=60, lattice=lattice, collision_rules=collision_rules)
lg_large.initialize_shear_flow(density=0.4)

print("Initial shear flow:")
lg_large.visualize_macroscopic(title='Macroscopic View: Initial Shear Flow')


=== Example 2: Larger System Macroscopic View ===
Initial shear flow:


In [9]:
# Evolve shear flow
print("\nAfter 50 steps:")
for _ in range(50):
    lg_large.step()
lg_large.visualize_macroscopic(title='Macroscopic View: After 50 Steps')


After 50 steps:


In [10]:
print("\n=== Example 3: Evolution with Obstacle (Flow Around Cylinder) ===")

# System with obstacle
lg_obstacle = LatticeGas(Nx=100, Ny=60, lattice=lattice, collision_rules=collision_rules)
lg_obstacle.initialize_obstacle(density=0.3, obstacle_center=(25, 30), obstacle_radius=8)

print("Initial state with obstacle:")
lg_obstacle.visualize_macroscopic(title='Flow Around Obstacle: Initial')


=== Example 3: Evolution with Obstacle (Flow Around Cylinder) ===
Initial state with obstacle:


In [11]:
# Evolve
print("\nAfter 100 steps:")
for _ in range(100):
    lg_obstacle.step()
lg_obstacle.visualize_macroscopic(title='Flow Around Obstacle: After 100 Steps')


After 100 steps:


In [12]:
# ===================================================================
# PART 4: TIME EVOLUTION AND CONSERVATION LAWS
# ===================================================================

print("\n=== Example 4: Verify Conservation Laws ===")

# Create a system and track conserved quantities
lg_conserve = LatticeGas(Nx=50, Ny=50, lattice=lattice, collision_rules=collision_rules)
lg_conserve.initialize_random(density=0.3)

num_steps = 100
times = []
masses = []
px_values = []
py_values = []

for t in range(num_steps):
    lg_conserve.step()
    
    # Record quantities
    times.append(t)
    masses.append(np.sum(lg_conserve.get_density()))
    px, py = lg_conserve.get_momentum()
    px_values.append(px)
    py_values.append(py)

# Plot conservation
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=['Total Mass vs Time', 'Total Momentum vs Time']
)

fig.add_trace(
    go.Scatter(x=times, y=masses, mode='lines', name='Mass',
               line=dict(color='blue', width=2)),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=times, y=px_values, mode='lines', name='p_x',
               line=dict(color='red', width=2)),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=times, y=py_values, mode='lines', name='p_y',
               line=dict(color='green', width=2)),
    row=2, col=1
)

fig.update_xaxes(title_text='Time Step', row=2, col=1)
fig.update_yaxes(title_text='Total Mass', row=1, col=1)
fig.update_yaxes(title_text='Total Momentum', row=2, col=1)

fig.update_layout(
    title_text='Conservation Laws in Lattice Gas',
    template='plotly_white',
    height=700,
    showlegend=True
)

fig.show()

print(f"Initial mass: {masses[0]:.1f}")
print(f"Final mass: {masses[-1]:.1f}")
print(f"Mass conservation: {abs(masses[-1] - masses[0]) < 1}")
print(f"\nInitial momentum: ({px_values[0]:.2f}, {py_values[0]:.2f})")
print(f"Final momentum: ({px_values[-1]:.2f}, {py_values[-1]:.2f})")


=== Example 4: Verify Conservation Laws ===


Initial mass: 4604.0
Final mass: 4604.0
Mass conservation: True

Initial momentum: (49.50, 66.68)
Final momentum: (49.50, 66.68)


In [13]:
# ===================================================================
# PART 5: COARSE-GRAINING (MICRO TO MACRO TRANSITION)
# ===================================================================

print("\n=== Example 5: Coarse-Graining Process ===")

class CoarseGrainer:
    """
    Coarse-grain the microscopic lattice gas to macroscopic fields.
    """
    
    def __init__(self, lattice_gas, coarse_factor=4):
        """
        Parameters:
        -----------
        lattice_gas : LatticeGas
            The microscopic simulation
        coarse_factor : int
            Spatial coarse-graining factor (average over coarse_factor x coarse_factor blocks)
        """
        self.lg = lattice_gas
        self.coarse_factor = coarse_factor
        
        self.Nx_coarse = lattice_gas.Nx // coarse_factor
        self.Ny_coarse = lattice_gas.Ny // coarse_factor
    
    def coarse_grain_density(self):
        """Average density over coarse blocks"""
        rho = self.lg.get_density()
        cf = self.coarse_factor
        
        rho_coarse = np.zeros((self.Nx_coarse, self.Ny_coarse))
        for i in range(self.Nx_coarse):
            for j in range(self.Ny_coarse):
                rho_coarse[i, j] = np.mean(rho[i*cf:(i+1)*cf, j*cf:(j+1)*cf])
        
        return rho_coarse
    
    def coarse_grain_velocity(self):
        """Average velocity over coarse blocks"""
        ux, uy = self.lg.get_velocity()
        cf = self.coarse_factor
        
        ux_coarse = np.zeros((self.Nx_coarse, self.Ny_coarse))
        uy_coarse = np.zeros((self.Nx_coarse, self.Ny_coarse))
        
        for i in range(self.Nx_coarse):
            for j in range(self.Ny_coarse):
                ux_coarse[i, j] = np.mean(ux[i*cf:(i+1)*cf, j*cf:(j+1)*cf])
                uy_coarse[i, j] = np.mean(uy[i*cf:(i+1)*cf, j*cf:(j+1)*cf])
        
        return ux_coarse, uy_coarse
    
    def visualize_comparison(self):
        """Compare microscopic and coarse-grained views"""
        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=['Microscopic Density', 'Coarse-Grained Density']
        )
        
        # Microscopic
        rho_micro = self.lg.get_density()
        fig.add_trace(
            go.Heatmap(z=rho_micro.T, colorscale='Viridis', showscale=False),
            row=1, col=1
        )
        
        # Coarse-grained
        rho_coarse = self.coarse_grain_density()
        fig.add_trace(
            go.Heatmap(z=rho_coarse.T, colorscale='Viridis', 
                      colorbar=dict(title='ρ')),
            row=1, col=2
        )
        
        fig.update_xaxes(title_text='x', row=1, col=1)
        fig.update_xaxes(title_text='x (coarse)', row=1, col=2)
        fig.update_yaxes(title_text='y', row=1, col=1)
        fig.update_yaxes(title_text='y (coarse)', row=1, col=2)
        
        fig.update_layout(
            title_text='Coarse-Graining: From Micro to Macro',
            template='plotly_white',
            height=500,
            width=1000
        )
        
        fig.show()

# Create a system and coarse-grain
lg_cg = LatticeGas(Nx=80, Ny=80, lattice=lattice, collision_rules=collision_rules)
lg_cg.initialize_random(density=0.3)

# Evolve to reach some structure
for _ in range(50):
    lg_cg.step()

# Coarse-grain
cg = CoarseGrainer(lg_cg, coarse_factor=4)
cg.visualize_comparison()


=== Example 5: Coarse-Graining Process ===


In [17]:
# ===================================================================
# PART 6: EMERGENT HYDRODYNAMICS
# ===================================================================

print("\n=== Example 6: Emergent Hydrodynamic Behavior ===")

def simulate_and_analyze(Nx, Ny, num_steps, init_type='shear'):
    """
    Run simulation and track macroscopic evolution.
    """
    lg = LatticeGas(Nx=Nx, Ny=Ny, lattice=lattice, collision_rules=collision_rules)
    
    if init_type == 'shear':
        lg.initialize_shear_flow(density=0.4)
    elif init_type == 'random':
        lg.initialize_random(density=0.3)
    
    # Track velocity profiles over time
    times_sample = [0, num_steps//4, num_steps//2, num_steps]
    velocity_profiles = []
    
    for t in range(num_steps + 1):
        if t in times_sample:
            ux, uy = lg.get_velocity()
            # Average velocity profile along y
            ux_profile = np.mean(ux, axis=0)
            velocity_profiles.append(ux_profile)
        
        if t < num_steps:
            lg.step()
    
    return times_sample, velocity_profiles

# Run simulation
Nx, Ny = 60, 60
num_steps = 200
times_sample, velocity_profiles = simulate_and_analyze(Nx, Ny, num_steps, init_type='shear')

# Plot velocity profile evolution
fig = go.Figure()

# Use plotly's built-in color sequences
import plotly.express as px
colors = px.colors.sequential.Viridis
n_profiles = len(velocity_profiles)

for i, (t, profile) in enumerate(zip(times_sample, velocity_profiles)):
    color = colors[int(i * len(colors) / n_profiles)]
    fig.add_trace(go.Scatter(
        x=np.arange(len(profile)),
        y=profile,
        mode='lines',
        name=f't = {t}',
        line=dict(color=color, width=2)
    ))

fig.update_layout(
    title='Evolution of Velocity Profile: Shear Flow Relaxation',
    xaxis_title='y position',
    yaxis_title='u_x (horizontal velocity)',
    template='plotly_white',
    height=500
)

fig.show()

print("Notice how the sharp velocity discontinuity diffuses over time")
print("This is viscous momentum diffusion emerging from particle collisions!")


=== Example 6: Emergent Hydrodynamic Behavior ===


Notice how the sharp velocity discontinuity diffuses over time
This is viscous momentum diffusion emerging from particle collisions!


In [15]:
# ===================================================================
# PART 7: SUMMARY VISUALIZATION
# ===================================================================

print("\n=== Example 7: Multi-Scale Summary ===")

# Create a comprehensive visualization showing multiple scales
lg_summary = LatticeGas(Nx=100, Ny=80, lattice=lattice, collision_rules=collision_rules)
lg_summary.initialize_obstacle(density=0.35, obstacle_center=(30, 40), obstacle_radius=10)

# Evolve
for _ in range(150):
    lg_summary.step()

# Create multi-panel figure
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Density Field (ρ)',
        'Velocity Magnitude (|u|)',
        'Horizontal Velocity (u_x)',
        'Vertical Velocity (u_y)'
    ],
    specs=[[{'type': 'heatmap'}, {'type': 'heatmap'}],
           [{'type': 'heatmap'}, {'type': 'heatmap'}]]
)

rho = lg_summary.get_density()
ux, uy = lg_summary.get_velocity()
u_mag = np.sqrt(ux**2 + uy**2)

# Density
fig.add_trace(
    go.Heatmap(z=rho.T, colorscale='Viridis', showscale=True, 
              colorbar=dict(x=0.46, len=0.4, y=0.75)),
    row=1, col=1
)

# Velocity magnitude
fig.add_trace(
    go.Heatmap(z=u_mag.T, colorscale='Plasma', showscale=True,
              colorbar=dict(x=1.02, len=0.4, y=0.75)),
    row=1, col=2
)

# u_x
fig.add_trace(
    go.Heatmap(z=ux.T, colorscale='RdBu', showscale=True, zmid=0,
              colorbar=dict(x=0.46, len=0.4, y=0.25)),
    row=2, col=1
)

# u_y
fig.add_trace(
    go.Heatmap(z=uy.T, colorscale='RdBu', showscale=True, zmid=0,
              colorbar=dict(x=1.02, len=0.4, y=0.25)),
    row=2, col=2
)

for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(title_text='x', row=i, col=j)
        fig.update_yaxes(title_text='y', row=i, col=j)

fig.update_layout(
    title_text='Lattice Gas: Macroscopic Fields from Microscopic Dynamics',
    template='plotly_white',
    height=800,
    width=1000
)

fig.show()

print("\nKey observations:")
print("1. Density field shows flow around obstacle")
print("2. Velocity magnitude shows wake formation")
print("3. Velocity components show vorticity and circulation")
print("4. All from simple Boolean particle rules!")


=== Example 7: Multi-Scale Summary ===



Key observations:
1. Density field shows flow around obstacle
2. Velocity magnitude shows wake formation
3. Velocity components show vorticity and circulation
4. All from simple Boolean particle rules!


In [16]:
# ===================================================================
# FINAL SUMMARY
# ===================================================================

print("\n" + "="*60)
print("SUMMARY: From Microdynamics to Macrodynamics")
print("="*60)
print("\nMICROSCOPIC LEVEL:")
print("  • Boolean particles (n_i ∈ {0,1})")
print("  • 6 velocity directions on hexagonal lattice")
print("  • Simple collision rules (2-body, 3-body)")
print("  • Conserve mass and momentum locally")
print("\nMESOSCOPIC LEVEL:")
print("  • Discrete time evolution: collision → propagation")
print("  • Statistical fluctuations at small scales")
print("  • Emergence of correlations")
print("\nMACROSCOPIC LEVEL:")
print("  • Continuous density field ρ(x,y,t)")
print("  • Continuous velocity field u(x,y,t)")
print("  • Obeys Navier-Stokes equations (in limit)")
print("  • Pressure from particle collisions")
print("  • Viscosity from momentum diffusion")
print("\nKEY INSIGHT:")
print("  Simple microscopic rules + conservation laws")
print("  → Complex macroscopic fluid behavior")
print("="*60)


SUMMARY: From Microdynamics to Macrodynamics

MICROSCOPIC LEVEL:
  • Boolean particles (n_i ∈ {0,1})
  • 6 velocity directions on hexagonal lattice
  • Simple collision rules (2-body, 3-body)
  • Conserve mass and momentum locally

MESOSCOPIC LEVEL:
  • Discrete time evolution: collision → propagation
  • Statistical fluctuations at small scales
  • Emergence of correlations

MACROSCOPIC LEVEL:
  • Continuous density field ρ(x,y,t)
  • Continuous velocity field u(x,y,t)
  • Obeys Navier-Stokes equations (in limit)
  • Pressure from particle collisions
  • Viscosity from momentum diffusion

KEY INSIGHT:
  Simple microscopic rules + conservation laws
  → Complex macroscopic fluid behavior
